In [1]:
import pandas as pd
import numpy as np
from chromadb import PersistentClient
from typing import List
import uuid
import ast

# from npl.npl import batch, batch_size

In [2]:
# Cargar datos
data = pd.read_parquet('productos_con_embeddings.parquet')
print(f"Datos cargados: {len(data)} registros")

Datos cargados: 41304 registros


In [3]:
data['categories_txt'] = data['categories'].apply(lambda x: ', '.join(ast.literal_eval(x) if isinstance(x, str) else x))

In [4]:
# Inicializar ChromaDB
client = PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="productos")

In [5]:
# df = data.iloc[:5000] # Para pruebas con pocas filas
df = data
len(df)

41304

In [6]:
embeddings = []
metadatas = []
documents = []
for _, row in df.iterrows():
    emb = row['embedding']
    embeddings.append(emb)

    metadata = {
            'producto': str(row['product_name']),
            'descripcion': str(row['descripcion_producto']),
            'precio': float(row['price']),
            'market': str(row['source']),
            'categorias': str(row['categories_txt']),
            'url': str(row['link'])
        }
    metadatas.append(metadata)

    document = f"Producto: {row['product_name']}. Descripción: {row['descripcion_producto']}. Categorías: {row['categories_txt']}"
    documents.append(document)


In [7]:
ids = [str(uuid.uuid4()) for _ in range(len(df))]
total_records = len(ids)
batch_size = 50
for i in range(0, total_records, batch_size):
    end_idx = min(i + batch_size, total_records)

    batch_ids = ids[i:end_idx]
    batch_embeddings = embeddings[i:end_idx]
    batch_metadatas = metadatas[i:end_idx]
    batch_documents = documents[i:end_idx]

    try:
        collection.add(
            ids=batch_ids,
            embeddings=batch_embeddings,
            metadatas=batch_metadatas,
            documents=batch_documents
        )
        print(f"Insertados registros {i} a {end_idx} de {total_records}")
    except Exception as e:
        print(f"Error en lote {i}-{end_idx}: {str(e)}")
        if batch_embeddings:
            print(f"Dimensión del embedding: {len(batch_embeddings[0])}")
        continue

Insertados registros 0 a 50 de 41304
Insertados registros 50 a 100 de 41304
Insertados registros 100 a 150 de 41304
Insertados registros 150 a 200 de 41304
Insertados registros 200 a 250 de 41304
Insertados registros 250 a 300 de 41304
Insertados registros 300 a 350 de 41304
Insertados registros 350 a 400 de 41304
Insertados registros 400 a 450 de 41304
Insertados registros 450 a 500 de 41304
Insertados registros 500 a 550 de 41304
Insertados registros 550 a 600 de 41304
Insertados registros 600 a 650 de 41304
Insertados registros 650 a 700 de 41304
Insertados registros 700 a 750 de 41304
Insertados registros 750 a 800 de 41304
Insertados registros 800 a 850 de 41304
Insertados registros 850 a 900 de 41304
Insertados registros 900 a 950 de 41304
Insertados registros 950 a 1000 de 41304
Insertados registros 1000 a 1050 de 41304
Insertados registros 1050 a 1100 de 41304
Insertados registros 1100 a 1150 de 41304
Insertados registros 1150 a 1200 de 41304
Insertados registros 1200 a 1250 de

In [8]:
collection.count()

41304

In [9]:
collection.peek(limit = 5)

{'ids': ['b33135fe-6f8c-446f-a3e8-aa062ff06cba',
  '75ca4304-d5ac-4c12-bc4d-c24f2cd4c2bc',
  'ed9f18f4-3098-4289-95c0-e4a8f5347b63',
  '56dab8b3-1a10-46a6-adbc-02cc757443a3',
  '23f5aa4f-1a33-4171-a53e-77ac1f4c0b07'],
 'embeddings': array([[ 0.00302135,  0.0149789 ,  0.01100438, ..., -0.01178116,
         -0.01896637,  0.00985863],
        [ 0.0056609 ,  0.00417483,  0.0042605 , ..., -0.00388487,
         -0.01560537, -0.01146678],
        [ 0.01129567, -0.00308374, -0.00845714, ..., -0.00745683,
         -0.00235299, -0.01175685],
        [ 0.01148252, -0.00785959, -0.00212493, ...,  0.00413437,
         -0.02597426, -0.01276276],
        [ 0.00063991, -0.01503273, -0.00202468, ..., -0.01631932,
         -0.00428975,  0.00244959]]),
 'documents': ['Producto: Agua Tónica BRITVIC Paquete 12un Lata 150ml. Descripción: .Agua tónica Marca: BRITVIC  Contenido: 12un de 150ml  .AGUASplazaVea presenta una gran variedad de Bebidas para que elijas tu favorita de acuerdo a diferentes situaciones 